In [252]:
import os
import sys
import torch 
import torch.nn as nn
from torch.optim import SGD
import sklearn
import numpy as np
import pandas as pd
from pprint import pprint
import jsonlines
import time

In [88]:
## paths
emb_dir = "/Users/weiw2/Temp/hf/data/embs"
txt_dir = "/Users/weiw2/Temp/hf/classified_txt/"
label_file = "/Users/weiw2/Temp/hf/data/ef_labels.csv"

In [4]:
## hyperparameters


In [93]:
## load embeddings
def load_avg_doc_embs(infile):
    """
        Get the CLS embedding of the last layer
        typically, the doc length is longer than the max_seq_len, e.g., 128
    """
    with open(infile) as fin:
        json = jsonlines.Reader(fin)
        all_cls_embs = []
        for line in json: ## a line is a chunk of the max_seq_length from this doc
            feats = line['features']
            feat_cls = feats[0] # [CLS] 
            final_layer = feat_cls['layers'][0]['values'] ## [CLS] embedding
            all_cls_embs.append(final_layer)
    ## get the mean of cls embeddings of all chunks from this doc
    all_cls_embs_tensor = torch.tensor(all_cls_embs)
    avg_doc_emb = torch.mean(all_cls_embs_tensor, dim=0)
    return avg_doc_emb

def get_pt_avg_emb(inpt_dir):
    docs = os.listdir(pid_dir)
    doc_num = len(docs)
    emb_dim = 768 ## hard coded. change it
    pt_all_embs_tensor = torch.zeros(doc_num, emb_dim)
    for idx in range(doc_num):
        emb_file = docs[idx]
        avg_doc_emb = load_avg_doc_embs(os.path.join(pid_dir, emb_file))
        pt_all_embs_tensor[idx] = avg_doc_emb
    return pt_all_embs_tensor

pt_emb_dict = {}
for src in os.listdir(emb_dir):
    src_dir = os.path.join(emb_dir, src)
    for pid in os.listdir(src_dir):
        pid_dir = os.path.join(src_dir, pid)
        pt_avg_emb = get_pt_avg_emb(pid_dir)
        pt_emb_dict[pid] = pt_avg_emb

In [102]:
## avg pt_emb_dict
for k, v in pt_emb_dict.items():
    pt_emb_dict[k] = torch.mean(v, dim=0)


In [46]:
with open("/Users/weiw2/Temp/hf/data/embs/NOTES-AMBULATORY_VISIT_SUMMARY/1220001226/20160923.jsonl") as fin:
    json = jsonlines.Reader(fin)
    for line in json:
        feats = line['features']
        feat_cls = feats[0] # [CLS]
        final_layer = feat_cls['layers'][0]


In [92]:
## load labels
label_df = pd.read_csv(label_file)
print(label_df.shape)
label_df.head()

(130774, 4)


,ID,VALUE,DATE,STUDY_ID
0,1,71.7,20171207,1221820821
1,2,35.0,20140228,1221176058
2,3,55.0,20140618,1221176058
3,4,45.0,20150605,1221748761
4,5,45.0,20151218,1221748761


In [197]:
pts_df = pd.DataFrame([int(id_) for id_ in list(pt_emb_dict.keys())], columns=["Patient_ID"])
sub_label_df = pd.merge(label_df, pts_df, left_on="STUDY_ID", right_on="Patient_ID")[label_df.columns]


def find_latest(x):
    x = x.sort_values(by='DATE', ascending=False)
    x = x.iloc[0,:]
    return x

sub_label_df = sub_label_df.groupby("STUDY_ID").apply(find_latest)
sub_label_df = sub_label_df.astype({"ID": int, "VALUE": float, "DATE": int, "STUDY_ID": int})
sub_label_df.head()

,ID,VALUE,DATE,STUDY_ID
STUDY_ID,,,,
1220000159,113646,35.0,20180606,1220000159
1220000216,28359,65.0,20170113,1220000216
1220000416,52434,45.0,20160902,1220000416
1220001054,92809,15.0,20180315,1220001054
1220001226,13344,35.0,20161127,1220001226


In [240]:
## Make label dict
pt_label_dict = {}
for pid in pt_emb_dict.keys():
    label = sub_label_df[sub_label_df["STUDY_ID"]==int(pid)]['VALUE'].tolist()
    if len(label) > 0:
        pt_label_dict[int(pid)] = float(label[0])
len(pt_label_dict)

19

In [ ]:
## make embedding tensor and label tensor

In [241]:
## regression
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.i2o = nn.Linear(input_size, output_size)

    def forward(self, inputs):
        output = self.i2o(inputs)
        return output    
    
pt_dim = 768
out_dim = 1
model = Model(pt_dim, out_dim)

In [244]:
## optimizer
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

In [248]:
criterion = nn.NLLLoss()

In [250]:
def train(input_tensor, target):    
    optimizer.zero_grad()
    output = model(input_tensor)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()        
    return output, loss.item() / input_line_tensor.size(0)

In [ ]:
def train(input_tensor, target):    
    optimizer.zero_grad()
    for pt, input_tensor in range(len(input_dict)):
        target = label_dict.get(pt)
        output = model(input_tensor)
        loss = criterion(output, target)
    loss.backward()
    optimizer.step()
        
    return output, loss.item() / input_line_tensor.size(0)

In [255]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [295]:
# n_iters = 100
# plot_every = 10
# start = time.time()

# # Keep track of losses for plotting
# current_loss = 0
# all_losses = []



# # len(pt_emb_dict), len(pt_label_dict)

# # for epic_idx in range(1, n_iters + 1):
    
# for pt, target in pt_label_dict.items():
#     input_tensor = pt_emb_dict.get(str(pt)) # torch.Size([768])
#     input_tensor = input_tensor.view(1, 1, -1)
#     target = torch.tensor([target], dtype=torch.long)
#     target = target.view(1, -1)
#     print(target.size())
#     output, loss = train(input_tensor, target)
#     current_loss += loss    
    
# #     output, loss = train()
    
    
# #     category, line, category_tensor, line_tensor = randomTrainingExample()
# #     output, loss = train(category_tensor, line_tensor)
# #     current_loss += loss

# #     # Print iter number, loss, name and guess
# #     if iter % print_every == 0:
# #         guess, guess_i = categoryFromOutput(output)
# #         correct = '✓' if guess == category else '✗ (%s)' % category
# #         print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

# #     # Add current loss avg to list of losses
# #     if iter % plot_every == 0:
# #         all_losses.append(current_loss / plot_every)
# #         current_loss = 0

In [279]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

In [280]:
diabetes = datasets.load_diabetes()
X = diabetes.data[:150]
y = diabetes.target[:150]
lasso = linear_model.Lasso()

In [284]:
pt_label_dict
pt_emb_dict = {}

(150,)

In [290]:
X = []
y = []
for pt, target in pt_label_dict.items():
    y.append(target)
    emb = pt_emb_dict.get(str(pt)).squeeze().numpy()
    X.append(emb)
    

In [291]:
X = np.array(X)
X.shape

(19, 768)

In [292]:
y = np.array(y)
y.shape

(19,)

In [294]:
cv_results = cross_validate(lasso, X, y, cv=5)
# sorted(cv_results.keys())
cv_results['test_score']

array([ 0.10029695, -6.44376761, -0.09643739, -2.19514003, -0.47890627])